# Retrain CFN Model (Colab)

This notebook retrains the Causal Fusion Network (CFN) with efficient techniques and saves the new weights to `models/cfn.pth`.


## Efficient techniques used
- Feature standardization (train-set mean/std)
- Class-imbalance weighting (positive class upweighting)
- Mixed precision on GPU (AMP)
- Cosine learning-rate schedule
- Gradient clipping
- Early stopping with validation loss


## 1. Setup
Mount Drive (optional) and install dependencies.


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# !pip install -r /content/drive/MyDrive/CausalX/requirements.txt


## 2. Set project paths
Update these paths to match your Drive or Colab workspace.


In [ ]:
from pathlib import Path

PROJECT_ROOT = Path('/content/drive/MyDrive/CausalX')
DATA_ROOT = PROJECT_ROOT / 'backend' / 'data'
PROCESSED_CSV = DATA_ROOT / 'processed' / 'causal_multimodal_dataset.csv'
MODEL_DIR = PROJECT_ROOT / 'backend' / 'models'


## 3. (Optional) Build the feature dataset
Run the batch feature extractor if the processed CSV does not exist or you updated feature definitions.


### Current feature set (must match the CSV)
The CFN training script expects the following AV features (plus jitter features). If you add or change features in the extractor, regenerate the CSV before training.

**AV features (baseline):**
- lip_variance, lip_mean, lip_std, lip_range
- lip_velocity_mean, lip_velocity_std
- av_correlation, av_lag_frames
- av_corr_05s_mean/std, av_corr_10s_mean/std, av_corr_20s_mean/std
- audio_rms_mean/std
- spectral_centroid_mean/std
- spectral_flux_mean/std
- mouth_flow_mean/std
- mouth_aspect_mean/std

**Embedding columns (optional V2):**
- tcn_visual_emb
- wav2vec_audio_emb

**Physical features:**
- jitter_mean, jitter_std

To train the embedding-aware model, run training with `--use-embeddings` and save `models/cfn_emb.pth`.


In [ ]:
# %cd /content/drive/MyDrive/CausalX/backend
# !python -m src.preprocessing.batch_feature_extractor


## 3a. (Optional) Balance the dataset (Option A)
If your dataset is highly imbalanced, downsample to the smallest class before training.


In [ ]:
# %cd /content/drive/MyDrive/CausalX/backend
# !python -m src.preprocessing.balance_dataset \
#   --input-csv data/processed/causal_multimodal_dataset.csv \
#   --output-csv data/processed/causal_multimodal_dataset_balanced.csv


## 4. Train the model
The script will save the best checkpoint to `models/cfn.pth`.


In [ ]:
# %cd /content/drive/MyDrive/CausalX/backend
# !python -m src.training.train_cfn \
#   --dataset-csv data/processed/causal_multimodal_dataset.csv \
#   --use-embeddings \
#   --epochs 30 \
#   --batch-size 128 \
#   --lr 1e-3 \
#   --weight-decay 1e-4
